## Set-up

Loading in packages and data

In [1]:
import os
import csv
import pandas as pd
import seaborn as sb
import numpy as np
from numpy import var, mean
import matplotlib.pyplot as plt
import random
import matplotlib.dates as mdates
import datetime as dt
import sklearn
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import scipy as spy
from scipy.stats import shapiro
import math
from math import sqrt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

os.chdir('directory') #directory removed



FileNotFoundError: [Errno 2] No such file or directory: 'directory'

In [ ]:
runmetrics = pd.read_csv('filename') #filename removed
list(runmetrics)

In [ ]:
len(runmetrics)

Defining functions

In [ ]:
def parseTime(timeString):
    saveTime = timeString[-12:]
    return(saveTime)

 ### UNUSED ###
def stringIndex(tempString, string_to_find):
    idx = []
    end = len(tempString)
    start = 0
    while start < len(tempString):
        tempIdx = teststring.find(string_to_find, start, end)
        if tempIdx >=0:
            idx.append(tempIdx)
            start = tempIdx + 1
        else:
            break
    return(idx)
### ###

def parsePersona(tempstring):
    start = (tempstring.find('"question12":')) + len('"question12":') + 2
    end = (tempstring.find('"question13"'))-3
    answer = tempstring[start:end]
    return(answer)

def parseID(tempstring):
    x = (tempstring.find('"id":')) + len('"id":') + 2
    y = (tempstring.find('"data"'))-3
    ID = tempstring[x:y]
    return(ID)

def definePersona(entry):
    if entry == 'My physical health depends on it' or entry == 'To tone, build, or maintain my current physique':
        persona = 'Goal Seeker'
    elif entry == 'To be outdoors':
        persona = 'Trail'
    elif entry == 'To run longer or faster':
        persona = 'Pacer'
    else:
        persona = 'Soul Runner'
    return(persona)

Converting time as a string into time format (removing date).

In [ ]:
tempTime = runmetrics.local_start_time.apply(parseTime)
runmetrics['time_of_day'] = tempTime
runmetrics.time_of_day = pd.to_datetime(runmetrics.time_of_day,format= '%H:%M:%S.%f' ).dt.time

In [ ]:
runmetrics.local_start_time

In [ ]:
#converting to datetime to see distribution of runs during day - date not preserved
tempDT = pd.to_datetime(runmetrics.time_of_day, format='%H:%M:%S')

#converting to datetime to see distribution of runs during day - date preserved
tempDT2 = pd.to_datetime(runmetrics.local_start_time, format = '%Y/%m/%d %H:%M:%S.%f')

In [ ]:
start_day = tempDT2.min()
finish_day = tempDT2.max()
print('Start = ', start_day)
print('Finish = ', finish_day)

challenge_length = finish_day - start_day
print('Difference = ', challenge_length)


In [ ]:
challenge_start = dt.datetime(2020, 12, 14)
challenge_end = dt.datetime(2021, 4, 2)

In [ ]:
runmetrics = runmetrics[(tempDT2 > challenge_start) & (tempDT2 < challenge_end)]
tempDT = tempDT[(tempDT2 > challenge_start) & (tempDT2 < challenge_end)]
tempDT2 = tempDT2[(tempDT2 > challenge_start) & (tempDT2 < challenge_end)]

In [ ]:
plt.figure(figsize = (9,6))
sb.histplot(tempDT)
plt.title('Time of Day for Runs')
;

In [ ]:
tempDT2.groupby(tempDT2.dt.hour).count().plot(kind="bar")

In [ ]:
plt.figure(figsize = (9,6))
sb.histplot(tempDT2)
plt.title('Runs Across days')
;

Plotting distribution of vertical stiffness

In [ ]:
runmetrics.average_kvert_running.describe()

In [ ]:
runmetrics.average_kvert_running.mean() + (3*runmetrics.average_kvert_running.std())
runmetrics.average_kvert_running.mean() - (3*runmetrics.average_kvert_running.std())

In [ ]:
#viewing distribution of vertical stiffness values
plt.figure(figsize = (9,6))
sb.histplot(runmetrics.average_kvert_running, binwidth = 1)
plt.title('Vertical Stiffness')
;

shapiro(runmetrics.average_kvert_running)

In [ ]:
#viewing extreme highs
plt.figure(figsize = (9,6))
sb.histplot(runmetrics.average_kvert_running[runmetrics.average_kvert_running > 40], binwidth = 1)
plt.title('Vertical Stiffness')
;

In [ ]:
#viewing extreme lows
plt.figure(figsize = (9,6))
sb.histplot(runmetrics.average_kvert_running[runmetrics.average_kvert_running < 20], binwidth = 1)
plt.title('Vertical Stiffness')
;

Plotting distribution of ground contact time

In [ ]:
runmetrics.average_gct_running.describe()

In [ ]:
plt.figure(figsize = (9,6))
sb.histplot(runmetrics.average_gct_running, binwidth = 1)
plt.title('Ground Contact Time')
;

In [ ]:
#viewing extreme highs
plt.figure(figsize = (9,6))
sb.histplot(runmetrics.average_gct_running[runmetrics.average_gct_running > 275], binwidth = 1)
plt.title('Ground Contact Time')
;

In [ ]:
#viewing extreme lows
plt.figure(figsize = (9,6))
sb.histplot(runmetrics.average_gct_running[runmetrics.average_gct_running < 200], binwidth = 1)
plt.title('Ground Contact Time')
;

Plotting distribution of cadence

In [ ]:
runmetrics.average_cadence_running.describe()

In [ ]:
plt.figure(figsize = (9,6))
sb.histplot(runmetrics.average_cadence_running, binwidth = 1)
plt.title('Cadence')
;

In [ ]:
plt.figure(figsize = (9,6))
sb.histplot(runmetrics.average_cadence_running[(runmetrics.average_cadence_running > 1) & (runmetrics.average_cadence_running < 125)], binwidth = 1)
plt.title('Cadence')
;

#runmetrics[(runmetrics.average_cadence_running >1) & (runmetrics.average_cadence_running < 50)].groupby('activity_type').size()

In [ ]:
runmetrics.plot(x = "time_of_day", y = "average_kvert", style=".")

In [ ]:
runmetrics['average_kvert'] = np.where(runmetrics['average_kvert'] == 0.0, np.NaN, runmetrics['average_kvert'])

runmetrics.plot(x = "time_of_day", y = "average_kvert", style=".")

Find proportion of runner personas in cohort.

In [ ]:
onboardSurvey = pd.read_csv('filename2.csv') #file name removed

runners = pd.DataFrame(columns = ['member_id', 'Persona_text'])

print(runners)
columns = list(runners)

data = []

In [ ]:
for row in onboardSurvey.index:
    values = [onboardSurvey.member_id[row], parsePersona(onboardSurvey.document[row])]
    pairs = zip(columns, values)
    tempDict = dict(pairs)
    data.append(tempDict)
    
runners = runners.append(data, ignore_index=True)

runners['Persona_group'] = list(map(definePersona, runners.Persona_text))
len(runners)
runners = runners.drop_duplicates()
len(runners)


In [ ]:
runners

In [ ]:
newDF = pd.DataFrame( data = {'member_id': runmetrics.member_id, 'time_of_day': tempDT})
newDF

In [ ]:
newDF2 = pd.DataFrame( data = {'member_id': runmetrics.member_id, 'time_of_day': tempDT2})
newDF2

In [ ]:
runmetricsV2 = runmetrics.merge(runners, how='inner', on='member_id')

In [ ]:
unqIDrun1 = list(set(runmetrics.member_id))
unqIDrun2 = list(set(runmetricsV2.member_id))

print(len(unqIDrun1))
print(len(unqIDrun2))

In [ ]:
len(runmetricsV2)


In [ ]:
# Unused but keeping for possible later use
#runmetricsV2 = pd.concat([runmetrics, runners.Persona_group, runners.Persona_text], axis = 1)

In [ ]:
ID_bool = runmetricsV2.member_id.duplicated()


for index, elem in enumerate(ID_bool):
    if elem == False:
        print(index)
        
temp = runmetricsV2[~ID_bool]

In [ ]:
temp.groupby('Persona_group').size()

In [ ]:
(temp.groupby('Persona_group').size()/len(temp)*100).plot(kind = 'bar', rot = 0)
plt.title('Proportion of Runner Personas (N = {})' .format(len(temp)))
plt.ylabel('Percentage')
plt.xlabel(None)

In [ ]:
runmetrics.groupby('member_id').size().sort_values()

In [ ]:
activity_task = pd.read_csv('filename3') #file name removed

In [ ]:
activity_task

In [ ]:
runmetricsV3 = runmetricsV2.merge(activity_task, how='left', on='activity_id')

In [ ]:
print('RunV1: ' , len(runmetrics))
print('RunV2: ' , len(runmetricsV2))
print('RunV3: ' , len(runmetricsV3))
print('unique activity: ', len(list(set(runmetricsV2.activity_id))))
print('unique activity: ', len(list(set(runmetricsV3.activity_id))))

In [ ]:
runmetricsV3.segment_name = runmetricsV3.segment_name.fillna('fun run')
runmetricsV3.task_id = runmetricsV3.task_id.fillna('no_task_id')

In [ ]:
print(set(runmetricsV3.segment_name))
print(set(runmetricsV3.task_id))


RQ4: How does time of day affect runmetrics? One hypothesis is that running in the dark could result in shorter stride length and slower speeds. Another hypothesis may be that vertical stiffness is greater in the morning. <br>
<br>
-Loop through all subjects <br>
-For eachs subject, plot each runmetrics against time of day (limit plotting to a few), find regression equation and correlation coefficient - store in another dataframe <br>
-Look at distributions of slopes and correlation coefficients

In [ ]:
#for elem in test_time:
#   new_time = elem.hour + (elem.minute/60)
 #   converted_times.append(new_time)

In [ ]:
def convertTime(list_of_times):
    converted_times = []
    for elem in list_of_times:
        new_time = elem.hour + (elem.minute/60)
        converted_times.append(new_time)
    return(converted_times)

In [ ]:
runmetricsV3['converted_times'] = pd.Series(convertTime(runmetricsV3.time_of_day))

In [ ]:
challenge_fun_run = runmetricsV3[(runmetricsV3.segment_name == 'fun run')]
len(challenge_fun_run)

In [ ]:
unq_subj = list(set(challenge_fun_run.member_id))

len(unq_subj)

In [ ]:
#runmetricsV3.average_kvert_running.isnull().sum()

In [ ]:
fig, axes = plt.subplots(3, 3, figsize = (12,9) )
plt.tight_layout(pad = 5)
count = -1
rowcount = 0
time_kvert_correlations = pd.DataFrame(columns = ['member_id','NumRuns', 'Slope', 'Intercept', 'Pearson', 'pval'])
for subj_id in unq_subj:
    print('ID: ', subj_id)
    rowcount += 1
    print('Count: ',rowcount)
    s_idx = challenge_fun_run.member_id == subj_id
    print(len(s_idx))
    if sum(s_idx) < 5:
        print('Skipping Subject - <5 runs')
    else:
        time_of_run =  challenge_fun_run.converted_times[s_idx]
        kvert_of_run = challenge_fun_run.average_kvert_running[s_idx]
        if time_of_run.isnull().sum() > 0 or kvert_of_run.isnull().sum() > 0:
            print('Skipping Subject - Null Values')
        else:
            count += 1
            reg = linear_model.LinearRegression().fit(time_of_run.values.reshape(-1,1), kvert_of_run.values.reshape(-1,1))
        #print(reg.coef_)
       # print(reg.intercept_)
       # print(spy.stats.pearsonr(time_of_run, kvert_of_run))
            try:    
                temp_slope = round(float(reg.coef_), 4)
                temp_intercept = round(float(reg.intercept_), 4)
                temp_pearson = spy.stats.pearsonr(time_of_run.values, kvert_of_run.values)
                num_runs = sum(s_idx)
                print([subj_id,num_runs, temp_slope, temp_intercept, temp_pearson[0],temp_pearson[1]])
                time_kvert_correlations.loc[count] =[subj_id,num_runs, temp_slope, temp_intercept, temp_pearson[0],temp_pearson[1]]
            except:
                print('Saving Data Failed')
            if count < 9:
                print('Graph Count:', count)
                row = count//3
                column = count%3
                axes[row,column].scatter(time_of_run,kvert_of_run)
            #axes[row,column].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
            #axes[row,column].tick_params(labelrotation=45)
        
            #plt.show()
        

In [ ]:
time_kvert_correlations

In [ ]:
sb.histplot(time_kvert_correlations.Slope)

In [ ]:
sb.histplot(time_kvert_correlations.Intercept)

In [ ]:
sb.histplot(time_kvert_correlations.Pearson, binwidth = 0.05)

In [ ]:
fig, axes = plt.subplots(3, 3, figsize = (12,9) )
plt.tight_layout(pad = 5)
count = -1
rowcount = 0
time_cadence_correlations = pd.DataFrame(columns = ['member_id','NumRuns', 'Slope', 'Intercept', 'Pearson', 'pval'])
for subj_id in unq_subj:
    print('ID: ', subj_id)
    rowcount += 1
    print('Count: ',rowcount)
    s_idx = challenge_fun_run.member_id == subj_id
    print(len(s_idx))
    if sum(s_idx) < 5:
        print('Skipping Subject - <5 runs')
    else:
        time_of_run =  challenge_fun_run.converted_times[s_idx]
        cadence_of_run = challenge_fun_run.average_cadence_running[s_idx]
        if time_of_run.isnull().sum() > 0 or cadence_of_run.isnull().sum() > 0:
            print('Skipping Subject - Null Values')
        else:
            count += 1
            reg = linear_model.LinearRegression().fit(time_of_run.values.reshape(-1,1), cadence_of_run.values.reshape(-1,1))
            try:    
                temp_slope = round(float(reg.coef_), 4)
                temp_intercept = round(float(reg.intercept_), 4)
                temp_pearson = spy.stats.pearsonr(time_of_run.values, cadence_of_run.values)
                num_runs = sum(s_idx)
                print([subj_id,num_runs, temp_slope, temp_intercept, temp_pearson[0],temp_pearson[1]])
                time_cadence_correlations.loc[count] =[subj_id,num_runs, temp_slope, temp_intercept, temp_pearson[0],temp_pearson[1]]
            except:
                print('Saving Data Failed')
            if count < 9:
                print('Graph Count:', count)
                row = count//3
                column = count%3
                axes[row,column].scatter(time_of_run,cadence_of_run)
   
        

In [ ]:
time_cadence_correlations

In [ ]:
sb.histplot(time_cadence_correlations.Slope)

In [ ]:
sb.histplot(time_cadence_correlations.Intercept)

In [ ]:
sb.histplot(time_cadence_correlations.Pearson, binwidth = 0.05)

In [ ]:
fig, axes = plt.subplots(3, 3, figsize = (12,9) )
plt.tight_layout(pad = 5)
count = -1
rowcount = 0
time_SL_correlations = pd.DataFrame(columns = ['member_id','NumRuns', 'Slope', 'Intercept', 'Pearson', 'pval'])
for subj_id in unq_subj:
    print('ID: ', subj_id)
    rowcount += 1
    print('Count: ',rowcount)
    s_idx = challenge_fun_run.member_id == subj_id
    print(len(s_idx))
    if sum(s_idx) < 5:
        print('Skipping Subject - <5 runs')
    else:
        time_of_run =  challenge_fun_run.converted_times[s_idx]
        SL_of_run = challenge_fun_run.average_stride_length_running[s_idx]
        if time_of_run.isnull().sum() > 0 or SL_of_run.isnull().sum() > 0:
            print('Skipping Subject - Null Values')
        else:
            count += 1
            reg = linear_model.LinearRegression().fit(time_of_run.values.reshape(-1,1), SL_of_run.values.reshape(-1,1))
        #print(reg.coef_)
       # print(reg.intercept_)
       # print(spy.stats.pearsonr(time_of_run, kvert_of_run))
            try:    
                temp_slope = round(float(reg.coef_), 4)
                temp_intercept = round(float(reg.intercept_), 4)
                temp_pearson = spy.stats.pearsonr(time_of_run.values, SL_of_run.values)
                num_runs = sum(s_idx)
                print([subj_id,num_runs, temp_slope, temp_intercept, temp_pearson[0],temp_pearson[1]])
                time_SL_correlations.loc[count] =[subj_id,num_runs, temp_slope, temp_intercept, temp_pearson[0],temp_pearson[1]]
            except:
                print('Saving Data Failed')
            if count < 9:
                print('Graph Count:', count)
                row = count//3
                column = count%3
                axes[row,column].scatter(time_of_run,SL_of_run)
            #axes[row,column].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
            #axes[row,column].tick_params(labelrotation=45)
        
            #plt.show()
        

In [ ]:
time_SL_correlations

In [ ]:
sb.histplot(time_SL_correlations.Slope)

In [ ]:
sb.histplot(time_SL_correlations.Intercept)

In [ ]:
sb.histplot(time_SL_correlations.Pearson, binwidth = 0.05)

In [ ]:
sb.histplot(time_kvert_correlations.NumRuns, binwidth = 1)
print('Avg # of runs: ', round(time_kvert_correlations.NumRuns.mean()))
print('SD # of runs: ', round(time_kvert_correlations.NumRuns.std()))


In [ ]:
avg_kvert_cor = round(time_kvert_correlations.Pearson.mean(),3)
SD_kvert_cor = round(time_kvert_correlations.Pearson.std(),3)
CI_kvert = [round(avg_kvert_cor - (1.96*SD_kvert_cor),3), round(avg_kvert_cor + (1.96*SD_kvert_cor),3)]

avg_cadence_cor = round(time_cadence_correlations.Pearson.mean(),3)
SD_cadence_cor = round(time_cadence_correlations.Pearson.std(),3)
CI_cadence = [round(avg_cadence_cor - (1.96*SD_cadence_cor),3), round(avg_cadence_cor + (1.96*SD_cadence_cor),3)]

avg_SL_cor = round(time_SL_correlations.Pearson.mean(),3)
SD_SL_cor = round(time_SL_correlations.Pearson.std(),3)
CI_SL = [round(avg_SL_cor - (1.96*SD_SL_cor),3), round(avg_SL_cor + (1.96*SD_SL_cor),3)]

print('Kvert Avg (CI): ', avg_kvert_cor, CI_kvert)
print('Cadence Avg (CI): ', avg_cadence_cor, CI_cadence)
print('SL Avg (CI): ', avg_SL_cor, CI_SL)

In [ ]:
subj_array = random.sample(unq_subj, 10)
subj_array

In [ ]:
sklearn.metrics.r2_score

In [ ]:
kvert_cor_table = pd.DataFrame(columns = ['iteration','NumRuns', 'Slope', 'Intercept', 'Pearson', 'pval'])
cadence_cor_table = pd.DataFrame(columns = ['iteration','NumRuns', 'Slope', 'Intercept', 'Pearson', 'pval'])
SL_cor_table = pd.DataFrame(columns = ['iteration','NumRuns', 'Slope', 'Intercept', 'Pearson', 'pval'])

iter_count = -1
while iter_count < 1000:
    iter_count += 1
    subj_array = random.sample(unq_subj, 10)
    temp_time = challenge_fun_run.converted_times[challenge_fun_run.member_id.isin(subj_array)]
    temp_kvert =  challenge_fun_run.average_kvert_running[challenge_fun_run.member_id.isin(subj_array)]
    temp_cadence =  challenge_fun_run.average_cadence_running[challenge_fun_run.member_id.isin(subj_array)]
    temp_SL = challenge_fun_run.average_stride_length_running[challenge_fun_run.member_id.isin(subj_array)]
    if temp_time.isnull().sum() > 0 or temp_kvert.isnull().sum() > 0 or temp_cadence.isnull().sum() > 0 or temp_SL.isnull().sum() > 0:
        iter_count = iter_count - 1
        print('Skipping Iteration - Null Values')
    else:
        #count += 1
        kvert_reg = linear_model.LinearRegression().fit(temp_time.values.reshape(-1,1), temp_kvert.values.reshape(-1,1))
        cadence_reg = linear_model.LinearRegression().fit(temp_time.values.reshape(-1,1), temp_cadence.values.reshape(-1,1))
        SL_reg = linear_model.LinearRegression().fit(temp_time.values.reshape(-1,1), temp_SL.values.reshape(-1,1))
        try:    
            temp_kvert_slope = round(float(kvert_reg.coef_), 4)
            temp_kvert_intercept = round(float(kvert_reg.intercept_), 4)
            temp_kvert_pearson = spy.stats.pearsonr(temp_time.values, temp_kvert.values)
            temp_cadence_slope = round(float(cadence_reg.coef_), 4)
            temp_cadence_intercept = round(float(cadence_reg.intercept_), 4)
            temp_cadence_pearson = spy.stats.pearsonr(temp_time.values, temp_cadence.values)
            temp_SL_slope = round(float(SL_reg.coef_), 4)
            temp_SL_intercept = round(float(SL_reg.intercept_), 4)
            temp_SL_pearson = spy.stats.pearsonr(temp_time.values, temp_SL.values)            
            num_runs = sum(challenge_fun_run.member_id.isin(subj_array))
            print(iter_count)
            kvert_cor_table.loc[iter_count] =[iter_count, num_runs, temp_kvert_slope, temp_kvert_intercept, temp_kvert_pearson[0],temp_kvert_pearson[1]]
            cadence_cor_table.loc[iter_count] =[iter_count, num_runs, temp_cadence_slope, temp_cadence_intercept, temp_cadence_pearson[0],temp_cadence_pearson[1]]
            SL_cor_table.loc[iter_count] =[iter_count, num_runs, temp_SL_slope, temp_SL_intercept, temp_SL_pearson[0],temp_SL_pearson[1]]
        except:
            print('Saving Data Failed')
    

In [ ]:
kvert_cor_table.Pearson**2

In [ ]:
sb.histplot(kvert_cor_table.Pearson, binwidth = 0.05)

In [ ]:
sb.histplot(cadence_cor_table.Pearson, binwidth = 0.05)

In [ ]:
sb.histplot(SL_cor_table.Pearson, binwidth = 0.05)

In [ ]:
print('Avg # runs: ', round(kvert_cor_table.NumRuns.mean()))
print('SD # runs: ', round(kvert_cor_table.NumRuns.std()))

In [ ]:
avg_kvert_cor = round(kvert_cor_table.Pearson.mean(),3)
SD_kvert_cor = round(kvert_cor_table.Pearson.std(),3)
CI_kvert = [round(avg_kvert_cor - (1.96*SD_kvert_cor),3), round(avg_kvert_cor + (1.96*SD_kvert_cor),3)]

avg_cadence_cor = round(cadence_cor_table.Pearson.mean(),3)
SD_cadence_cor = round(cadence_cor_table.Pearson.std(),3)
CI_cadence = [round(avg_cadence_cor - (1.96*SD_cadence_cor),3), round(avg_cadence_cor + (1.96*SD_cadence_cor),3)]

avg_SL_cor = round(SL_cor_table.Pearson.mean(),3)
SD_SL_cor = round(SL_cor_table.Pearson.std(),3)
CI_SL = [round(avg_SL_cor - (1.96*SD_SL_cor),3), round(avg_SL_cor + (1.96*SD_SL_cor),3)]

print('Kvert Avg (CI): ', avg_kvert_cor, CI_kvert)
print('Cadence Avg (CI): ', avg_cadence_cor, CI_cadence)
print('SL Avg (CI): ', avg_SL_cor, CI_SL)


In [ ]:
tempDT

In [ ]:
temp_challenge_time_of_day = pd.to_datetime(challenge_fun_run.time_of_day, format='%H:%M:%S')
plt.figure(figsize = (12,8))
sb.histplot(temp_challenge_time_of_day, bins = 48)

In [ ]:
tenAM = dt.datetime(1900, 1, 1, hour = 10, minute = 0, second = 0)
elevenAM = dt.datetime(1900, 1, 1, hour = 11, minute = 0, second = 0)
noon = dt.datetime(1900, 1,1, hour = 12, minute = 0, second = 0)
twoPM = dt.datetime(1900, 1, 1, hour = 14, minute = 0, second = 0)
sevenAM = dt.datetime(1900, 1, 1, hour = 7, minute = 0, second = 0)
sevenPM = dt.datetime(1900, 1, 1, hour = 19, minute = 0, second = 0)
sixPM = dt.datetime(1900, 1, 1, hour = 18, minute = 0, second = 0)
threePM = dt.datetime(1900, 1, 1, hour = 15, minute = 0, second = 0)



In [ ]:
early_morning = challenge_fun_run[temp_challenge_time_of_day <= sevenAM]
morning = challenge_fun_run[(temp_challenge_time_of_day > sevenAM) & (temp_challenge_time_of_day <= elevenAM)]
midday = challenge_fun_run[(temp_challenge_time_of_day > elevenAM) & (temp_challenge_time_of_day <= threePM)]
late_afternoon = challenge_fun_run[(temp_challenge_time_of_day > threePM) & (temp_challenge_time_of_day <= sevenPM)]
night = challenge_fun_run[(temp_challenge_time_of_day > sevenPM)]

In [ ]:
def define_time_of_day(converted_time):
    if converted_time <= 7:
        period_of_day = 'Early Morning'
    elif 7 < converted_time <= 11:
        period_of_day = 'Morning'
    elif 11 < converted_time <= 15:
        period_of_day = 'Mid-day'  
    elif 15 < converted_time <= 19:
        period_of_day = 'Evening'
    elif converted_time > 19:
        period_of_day = 'Night'
    return(period_of_day)

In [ ]:
challenge_fun_run['Day_period'] = list(map(define_time_of_day, challenge_fun_run.converted_times))

In [ ]:
challenge_fun_run

In [ ]:
kvert_anova_model = ols('average_kvert_running ~ C(Day_period)', data = challenge_fun_run).fit()
kvert_anova_table = sm.stats.anova_lm(kvert_anova_model, typ=2)
kvert_anova_table

print("Early Morning mean kvert: ", challenge_fun_run.average_kvert_running[challenge_fun_run.Day_period == 'Early Morning'].mean(),
     ';SD: ', challenge_fun_run.average_kvert_running[challenge_fun_run.Day_period == 'Early Morning'].std(),
     '; # runs: ', sum(challenge_fun_run.Day_period == 'Early Morning'))
print("Morning mean kvert: ", challenge_fun_run.average_kvert_running[challenge_fun_run.Day_period == 'Morning'].mean(),
     ';SD: ', challenge_fun_run.average_kvert_running[challenge_fun_run.Day_period == 'Morning'].std(),
     ';# runs: ', sum(challenge_fun_run.Day_period == 'Morning'))
print("Midday mean kvert: ", challenge_fun_run.average_kvert_running[challenge_fun_run.Day_period == 'Mid-day'].mean(),
     ';SD: ', challenge_fun_run.average_kvert_running[challenge_fun_run.Day_period == 'Mid-day'].std(),
     ';# runs: ', sum(challenge_fun_run.Day_period == 'Mid-day'))
print("Afternoon mean kvert: ", challenge_fun_run.average_kvert_running[challenge_fun_run.Day_period == 'Evening'].mean(),
     ';SD: ', challenge_fun_run.average_kvert_running[challenge_fun_run.Day_period == 'Evening'].std(),
     ';# runs: ', sum(challenge_fun_run.Day_period == 'Evening'))
print("Night mean kvert: ", challenge_fun_run.average_kvert_running[challenge_fun_run.Day_period == 'Night'].mean(),
     ';SD: ', challenge_fun_run.average_kvert_running[challenge_fun_run.Day_period == 'Night'].std(),
     ';# runs: ', sum(challenge_fun_run.Day_period == 'Night'))

tukey_kvert_df = challenge_fun_run.filter(items=['average_kvert_running', 'Day_period']).dropna()
#mc_kvert = multi.MultiComparison(tukey_kvert_df['average_kvert_running'], tukey_kvert_df['Day_period'])
#kvert_tukey_result = mc_kvert.tukeyhsd()
kvert_tukey = pairwise_tukeyhsd(endog = tukey_kvert_df.average_kvert_running, groups = tukey_kvert_df.Day_period, alpha = 0.05)
print(kvert_tukey)

In [ ]:
# function to calculate Cohen's d for independent samples - from Machine Learning Mastery
def cohend(d1, d2):
    # calculate the size of samples
    n1, n2 = len(d1), len(d2)
    # calculate the variance of the samples
    s1, s2 = var(d1, ddof=1), var(d2, ddof=1)
    # calculate the pooled standard deviation
    s = sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
    # calculate the means of the samples
    u1, u2 = mean(d1), mean(d2)
    # calculate the effect size
    return (u1 - u2) / s

In [ ]:
kvert_Emorn_morn_d = cohend(tukey_kvert_df.average_kvert_running[tukey_kvert_df.Day_period == 'Early Morning'],
                      tukey_kvert_df.average_kvert_running[tukey_kvert_df.Day_period == 'Morning'])
kvert_Emorn_evening_d = cohend(tukey_kvert_df.average_kvert_running[tukey_kvert_df.Day_period == 'Early Morning'],
                      tukey_kvert_df.average_kvert_running[tukey_kvert_df.Day_period == 'Evening'])
kvert_Midday_evening_d = cohend(tukey_kvert_df.average_kvert_running[tukey_kvert_df.Day_period == 'Mid-day'],
                      tukey_kvert_df.average_kvert_running[tukey_kvert_df.Day_period == 'Evening'])
kvert_evening_night_d = cohend(tukey_kvert_df.average_kvert_running[tukey_kvert_df.Day_period == 'Evening'],
                      tukey_kvert_df.average_kvert_running[tukey_kvert_df.Day_period == 'Night'])

print('Early Morning - Morning: %.3f' % kvert_Emorn_morn_d)
print('Early Morning - Evening: %.3f' % kvert_Emorn_evening_d)
print('Midday - Evening: %.3f' % kvert_Midday_evening_d)
print('Evening - Night: %.3f' % kvert_evening_night_d)



In [ ]:
cadence_anova_model = ols('average_cadence_running ~ C(Day_period)', data = challenge_fun_run).fit()
cadence_anova_table = sm.stats.anova_lm(cadence_anova_model, typ=2)
cadence_anova_table

print("Early Morning mean cadence: ", challenge_fun_run.average_cadence_running[challenge_fun_run.Day_period == 'Early Morning'].mean(),
     ';SD: ', challenge_fun_run.average_cadence_running[challenge_fun_run.Day_period == 'Early Morning'].std(),
     '; # runs: ', sum(challenge_fun_run.Day_period == 'Early Morning'))
print("Morning mean cadence: ", challenge_fun_run.average_cadence_running[challenge_fun_run.Day_period == 'Morning'].mean(),
     ';SD: ', challenge_fun_run.average_cadence_running[challenge_fun_run.Day_period == 'Morning'].std(),
     ';# runs: ', sum(challenge_fun_run.Day_period == 'Morning'))
print("Midday mean cadence: ", challenge_fun_run.average_cadence_running[challenge_fun_run.Day_period == 'Mid-day'].mean(),
     ';SD: ', challenge_fun_run.average_cadence_running[challenge_fun_run.Day_period == 'Mid-day'].std(),
     ';# runs: ', sum(challenge_fun_run.Day_period == 'Mid-day'))
print("Afternoon mean cadence: ", challenge_fun_run.average_cadence_running[challenge_fun_run.Day_period == 'Evening'].mean(),
     ';SD: ', challenge_fun_run.average_cadence_running[challenge_fun_run.Day_period == 'Evening'].std(),
     ';# runs: ', sum(challenge_fun_run.Day_period == 'Evening'))
print("Night mean cadence: ", challenge_fun_run.average_cadence_running[challenge_fun_run.Day_period == 'Night'].mean(),
     ';SD: ', challenge_fun_run.average_cadence_running[challenge_fun_run.Day_period == 'Night'].std(),
     ';# runs: ', sum(challenge_fun_run.Day_period == 'Night'))

tukey_cadence_df = challenge_fun_run.filter(items=['average_cadence_running', 'Day_period']).dropna()
#mc_cadence = multi.MultiComparison(tukey_cadence_df['average_cadence_running'], tukey_cadence_df['Day_period'])
#cadence_tukey_result = mc_cadence.tukeyhsd()
cadence_tukey = pairwise_tukeyhsd(endog = tukey_cadence_df.average_cadence_running, groups = tukey_cadence_df.Day_period, alpha = 0.05)
print(cadence_tukey)

In [ ]:
cadence_Emorn_morn_d = cohend(tukey_cadence_df.average_cadence_running[tukey_cadence_df.Day_period == 'Early Morning'],
                      tukey_cadence_df.average_cadence_running[tukey_cadence_df.Day_period == 'Morning'])
cadence_Emorn_midday_d = cohend(tukey_cadence_df.average_cadence_running[tukey_cadence_df.Day_period == 'Early Morning'],
                      tukey_cadence_df.average_cadence_running[tukey_cadence_df.Day_period == 'Mid-day'])
cadence_Emorn_evening_d = cohend(tukey_cadence_df.average_cadence_running[tukey_cadence_df.Day_period == 'Early Morning'],
                      tukey_cadence_df.average_cadence_running[tukey_cadence_df.Day_period == 'Evening'])
cadence_Emorn_night_d = cohend(tukey_cadence_df.average_cadence_running[tukey_cadence_df.Day_period == 'Early Morning'],
                      tukey_cadence_df.average_cadence_running[tukey_cadence_df.Day_period == 'Night'])
cadence_morning_midday_d = cohend(tukey_cadence_df.average_cadence_running[tukey_cadence_df.Day_period == 'Morning'],
                      tukey_cadence_df.average_cadence_running[tukey_cadence_df.Day_period == 'Mid-day'])
cadence_morning_evening_d = cohend(tukey_cadence_df.average_cadence_running[tukey_cadence_df.Day_period == 'Morning'],
                      tukey_cadence_df.average_cadence_running[tukey_cadence_df.Day_period == 'Evening'])
cadence_morning_night_d = cohend(tukey_cadence_df.average_cadence_running[tukey_cadence_df.Day_period == 'Morning'],
                      tukey_cadence_df.average_cadence_running[tukey_cadence_df.Day_period == 'Night'])
cadence_midday_evening_d = cohend(tukey_cadence_df.average_cadence_running[tukey_cadence_df.Day_period == 'Mid-day'],
                      tukey_cadence_df.average_cadence_running[tukey_cadence_df.Day_period == 'Evening'])
cadence_midday_night_d = cohend(tukey_cadence_df.average_cadence_running[tukey_cadence_df.Day_period == 'Mid-day'],
                      tukey_cadence_df.average_cadence_running[tukey_cadence_df.Day_period == 'Night'])
cadence_evening_night_d = cohend(tukey_cadence_df.average_cadence_running[tukey_cadence_df.Day_period == 'Evening'],
                      tukey_cadence_df.average_cadence_running[tukey_cadence_df.Day_period == 'Night'])

print('Early Morning - Morning: %.3f' % cadence_Emorn_morn_d)
print('Early Morning - Midday: %.3f' % cadence_Emorn_midday_d)
print('Early Morning - Evening: %.3f' % cadence_Emorn_evening_d)
print('Early Morning - Night: %.3f' % cadence_Emorn_night_d)
print('Morning - Midday: %.3f' % cadence_morning_midday_d)
print('Morning - Evening: %.3f' % cadence_morning_evening_d)
print('Morning - Night: %.3f' % cadence_morning_night_d)
print('Midday - Evening: %.3f' % cadence_midday_evening_d)
print('Midday - Night: %.3f' % cadence_midday_night_d)
print('Evening - Night: %.3f' % cadence_evening_night_d)

In [ ]:
stride_length_anova_model = ols('average_stride_length_running ~ C(Day_period)', data = challenge_fun_run).fit()
stride_length_anova_table = sm.stats.anova_lm(stride_length_anova_model, typ=2)
stride_length_anova_table

print("Early Morning mean stride_length: ", challenge_fun_run.average_stride_length_running[challenge_fun_run.Day_period == 'Early Morning'].mean(),
     ';SD: ', challenge_fun_run.average_stride_length_running[challenge_fun_run.Day_period == 'Early Morning'].std(),
     '; # runs: ', sum(challenge_fun_run.Day_period == 'Early Morning'))
print("Morning mean stride_length: ", challenge_fun_run.average_stride_length_running[challenge_fun_run.Day_period == 'Morning'].mean(),
     ';SD: ', challenge_fun_run.average_stride_length_running[challenge_fun_run.Day_period == 'Morning'].std(),
     ';# runs: ', sum(challenge_fun_run.Day_period == 'Morning'))
print("Midday mean stride_length: ", challenge_fun_run.average_stride_length_running[challenge_fun_run.Day_period == 'Mid-day'].mean(),
     ';SD: ', challenge_fun_run.average_stride_length_running[challenge_fun_run.Day_period == 'Mid-day'].std(),
     ';# runs: ', sum(challenge_fun_run.Day_period == 'Mid-day'))
print("Afternoon mean stride_length: ", challenge_fun_run.average_stride_length_running[challenge_fun_run.Day_period == 'Evening'].mean(),
     ';SD: ', challenge_fun_run.average_stride_length_running[challenge_fun_run.Day_period == 'Evening'].std(),
     ';# runs: ', sum(challenge_fun_run.Day_period == 'Evening'))
print("Night mean stride_length: ", challenge_fun_run.average_stride_length_running[challenge_fun_run.Day_period == 'Night'].mean(),
     ';SD: ', challenge_fun_run.average_stride_length_running[challenge_fun_run.Day_period == 'Night'].std(),
     ';# runs: ', sum(challenge_fun_run.Day_period == 'Night'))

tukey_stride_length_df = challenge_fun_run.filter(items=['average_stride_length_running', 'Day_period']).dropna()
#mc_stride_length = multi.MultiComparison(tukey_stride_length_df['average_stride_length_running'], tukey_stride_length_df['Day_period'])
#stride_length_tukey_result = mc_stride_length.tukeyhsd()
stride_length_tukey = pairwise_tukeyhsd(endog = tukey_stride_length_df.average_stride_length_running, groups = tukey_stride_length_df.Day_period, alpha = 0.05)
print(stride_length_tukey)

In [ ]:
stride_length_Emorn_morn_d = cohend(tukey_stride_length_df.average_stride_length_running[tukey_stride_length_df.Day_period == 'Early Morning'],
                      tukey_stride_length_df.average_stride_length_running[tukey_stride_length_df.Day_period == 'Morning'])
stride_length_Emorn_midday_d = cohend(tukey_stride_length_df.average_stride_length_running[tukey_stride_length_df.Day_period == 'Early Morning'],
                      tukey_stride_length_df.average_stride_length_running[tukey_stride_length_df.Day_period == 'Mid-day'])
stride_length_Emorn_evening_d = cohend(tukey_stride_length_df.average_stride_length_running[tukey_stride_length_df.Day_period == 'Early Morning'],
                      tukey_stride_length_df.average_stride_length_running[tukey_stride_length_df.Day_period == 'Evening'])
stride_length_Emorn_night_d = cohend(tukey_stride_length_df.average_stride_length_running[tukey_stride_length_df.Day_period == 'Early Morning'],
                      tukey_stride_length_df.average_stride_length_running[tukey_stride_length_df.Day_period == 'Night'])
stride_length_morning_midday_d = cohend(tukey_stride_length_df.average_stride_length_running[tukey_stride_length_df.Day_period == 'Morning'],
                      tukey_stride_length_df.average_stride_length_running[tukey_stride_length_df.Day_period == 'Mid-day'])
stride_length_morning_evening_d = cohend(tukey_stride_length_df.average_stride_length_running[tukey_stride_length_df.Day_period == 'Morning'],
                      tukey_stride_length_df.average_stride_length_running[tukey_stride_length_df.Day_period == 'Evening'])
stride_length_morning_night_d = cohend(tukey_stride_length_df.average_stride_length_running[tukey_stride_length_df.Day_period == 'Morning'],
                      tukey_stride_length_df.average_stride_length_running[tukey_stride_length_df.Day_period == 'Night'])
stride_length_midday_evening_d = cohend(tukey_stride_length_df.average_stride_length_running[tukey_stride_length_df.Day_period == 'Mid-day'],
                      tukey_stride_length_df.average_stride_length_running[tukey_stride_length_df.Day_period == 'Evening'])
stride_length_midday_night_d = cohend(tukey_stride_length_df.average_stride_length_running[tukey_stride_length_df.Day_period == 'Mid-day'],
                      tukey_stride_length_df.average_stride_length_running[tukey_stride_length_df.Day_period == 'Night'])
stride_length_evening_night_d = cohend(tukey_stride_length_df.average_stride_length_running[tukey_stride_length_df.Day_period == 'Evening'],
                      tukey_stride_length_df.average_stride_length_running[tukey_stride_length_df.Day_period == 'Night'])

print('Early Morning - Morning: %.3f' % stride_length_Emorn_morn_d)
print('Early Morning - Midday: %.3f' % stride_length_Emorn_midday_d)
print('Early Morning - Evening: %.3f' % stride_length_Emorn_evening_d)
print('Early Morning - Night: %.3f' % stride_length_Emorn_night_d)
print('Morning - Midday: %.3f' % stride_length_morning_midday_d)
print('Morning - Evening: %.3f' % stride_length_morning_evening_d)
print('Morning - Night: %.3f' % stride_length_morning_night_d)
print('Midday - Evening: %.3f' % stride_length_midday_evening_d)
print('Midday - Night: %.3f' % stride_length_midday_night_d)
print('Evening - Night: %.3f' % stride_length_evening_night_d)

In [ ]:
runmetricsV3.to_csv('runmetricsV3.csv')